In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import mysql.connector
from mysql.connector import Error


app = Flask(__name__)

def get_db_connection():
    conn = mysql.connector.connect(
        host="localhost",
        user="admin",
        password="helloLayka",
        database="test123"
    )
    return conn



@app.route('/configuration')
def configuration():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)

    # Fetching all rows for OWNER, CATEGORY, DOC_STATE, REVIEW_STATE
    cursor.execute('SELECT * FROM OWNER')
    owner_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM CATEGORY')
    category_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM DOC_STATE')
    doc_state_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM REVIEW_STATE')
    review_state_rows = cursor.fetchall()

    # Query to join DOCUMENT with OWNER and CATEGORY tables
    cursor.execute('''
        SELECT DOCUMENT.id, DOCUMENT.name, DOCUMENT.initials, OWNER.name AS owner_name, CATEGORY.name AS category_name, DOCUMENT.owner_id, DOCUMENT.category_id
        FROM DOCUMENT
        JOIN OWNER ON DOCUMENT.owner_id = OWNER.id
        JOIN CATEGORY ON DOCUMENT.category_id = CATEGORY.id
    ''')
    document_rows = cursor.fetchall()

    # Fetching lists for dropdowns in the DOCUMENT table
    cursor.execute('SELECT id, name FROM OWNER')
    owner_list = cursor.fetchall()

    cursor.execute('SELECT id, name FROM CATEGORY')
    category_list = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('configuration.html',
                           owner_rows=owner_rows, 
                           category_rows=category_rows, 
                           doc_state_rows=doc_state_rows, 
                           review_state_rows=review_state_rows,
                           document_rows=document_rows,
                           owner_list=owner_list,
                           category_list=category_list)

@app.route('/modify/<table_name>/<int:row_id>', methods=['POST'])
def modify_row(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Build the update query dynamically based on the form fields
    update_fields = ', '.join([f"{key} = %s" for key in request.form.keys()])
    values = list(request.form.values())

    cursor.execute(f"UPDATE {table_name} SET {update_fields} WHERE id = %s", values + [row_id])
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('configuration'))



@app.route('/delete/<table_name>/<int:row_id>', methods=['POST'])
def delete_row(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute(f'DELETE FROM {table_name} WHERE id = %s', (row_id,))
        conn.commit()
    except Error as e:
        # Check if the error is due to a foreign key constraint
        if e.errno == 1451:  # MySQL error code for foreign key constraint failure
            cursor.close()
            conn.close()
            return f"Cannot delete this item because it is still in use. The {table_name} entry with ID {row_id} could not be deleted because it is referenced by another table.", 400
        else:
            raise e  # Re-raise if it's another kind of error
    
    cursor.close()
    conn.close()
    
    return redirect(url_for('configuration'))

@app.route('/add_row/<table_name>', methods=['POST'])
def add_row(table_name):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Extract the form data, excluding the id field since it's usually auto-incremented
    keys = ', '.join(request.form.keys())
    placeholders = ', '.join(['%s'] * len(request.form))
    values = list(request.form.values())

    cursor.execute(f"INSERT INTO {table_name} ({keys}) VALUES ({placeholders})", values)
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('configuration'))






if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.30:5001
Press CTRL+C to quit
192.168.1.18 - - [19/Aug/2024 21:32:21] "GET /configuration HTTP/1.1" 200 -
192.168.1.18 - - [19/Aug/2024 21:32:22] "GET /configuration HTTP/1.1" 200 -
192.168.1.18 - - [19/Aug/2024 21:32:25] "POST /delete/OWNER/4 HTTP/1.1" 400 -
[2024-08-19 21:33:42,333] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.11/site-packages/mysql/connector/connection_cext.py", line 705, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Table 'test123.your_table_name' doesn't exist

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/flask/app